In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os
import pickle


In [2]:

batch_size = 32
learning_rate = 0.001
num_epochs = 10
model_save_path = "fruit_model.pth"
labels_save_path = "fruit_labels.pkl"

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [3]:

# Data transforms
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [4]:

# Load datasets
train_dataset = datasets.ImageFolder(root=r'C:\Users\kara\Desktop\Project\fruits-360/Train', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.ImageFolder(root=r'C:\Users\kara\Desktop\Project\fruits-360/Test', transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [5]:
# Extract and save class labels
class_labels = train_dataset.classes
num_classes = len(class_labels)
print(f"Number of classes: {num_classes}")
print(f"Class labels: {class_labels}")

with open(labels_save_path, 'wb') as f:
    pickle.dump(class_labels, f)
print(f"Class labels saved to {labels_save_path}")

Number of classes: 31
Class labels: ['آناناس', 'اسفناج', 'انار', 'انگور', 'بادمجان', 'تربچه', 'خیار', 'ذرت', 'زنجبیل', 'سویا', 'سیب', 'سیب زمینی', 'سیر', 'شلغم', 'فلفل', 'فلفل دلمه', 'لیمو', 'مانگو', 'موز', 'نخود فرنگی', 'هندوانه', 'هویج', 'پرتغال', 'پیاز', 'چغندر', 'کاهو', 'کلم', 'کیوی', 'گل کلم', 'گلابی', 'گوجه']
Class labels saved to fruit_labels2.pkl


In [6]:

# Define the model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))  # Adjust output for number of classes
model = model.to(device)


c:\Users\kara\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\kara\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [8]:

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")


c:\Users\kara\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/10], Loss: 1.7865
Epoch [2/10], Loss: 0.9020
Epoch [3/10], Loss: 0.6979
Epoch [4/10], Loss: 0.5161
Epoch [5/10], Loss: 0.3318
Epoch [6/10], Loss: 0.2710
Epoch [7/10], Loss: 0.2237
Epoch [8/10], Loss: 0.2581
Epoch [9/10], Loss: 0.2140
Epoch [10/10], Loss: 0.1314


In [9]:

# Evaluate on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 92.02%


In [10]:
# Save the trained model
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to fruit_identifier_fruits362.pth
